# Fully-Connected Neural Network Training

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import re
import pandas as pd

In [ ]:
%load_ext tensorboard
%load_ext autoreload

### Load code for this project

In [ ]:
import fcnn.train
import fcnn.eval
import data_processing.data as dp
%autoreload 1
%aimport fcnn.train
%aimport fcnn.eval

In [ ]:
path_data = './data_processing/voxels/'
train, _ = dp.load_discretized_data(path_data, categorical=False, binary=True, normalize=True)

In [ ]:
train = (train[0].toarray(), train[1])

In [ ]:
!rm -rf fcnn/logs/*

In [ ]:
%%time
fcnn.train.train(train=train, 
                log_dir='fcnn/logs/',
                hidden_layers=[128],
                validation_split=0.15,
                lr=1e-3, 
                decay=0.,
                examples_limit=-1,
                epochs=100, 
                batch_size=32,
                seed=71,
                use_dropout=True,
                dropout=0.5,
               )

In [ ]:
%tensorboard --logdir fcnn/logs/ --port 6008

In [ ]:
fcnn.eval.eval(model_file='fcnn/logs/nodes128_dropoutTrue_lr0.001_decay0.0_samples-1/20200826-093241/saved_model.h5', data=train, name='FCNN')